In [ ]:
import pandas as pd
import torch
from transformers import pipeline
from bs4 import BeautifulSoup
from multiprocessing import Pool

# ---- CẤU HÌNH ----
INPUT_FILE = "data.xlsx"  # File dữ liệu đầu vào (CSV hoặc Excel)
OUTPUT_FILE = "summary_results.json"  # File lưu kết quả
MAX_WORDS_PER_CHUNK = 500  # Giới hạn từ mỗi đoạn tóm tắt

# ---- KIỂM TRA GPU ----
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔹 Đang chạy trên: {device}")

# ---- LOAD MÔ HÌNH TÓM TẮT ----
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device == "cuda" else -1)

# ---- HÀM TÓM TẮT ----
def summarize_text(text, max_length=150, min_length=50):
    """Tóm tắt một đoạn văn bản"""
    return summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

def extract_text(html_content):
    """Lấy văn bản sạch từ nội dung HTML"""
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text(separator=" ", strip=True)

def chunk_text(text, max_words=MAX_WORDS_PER_CHUNK):
    """Chia nhỏ văn bản dài để tóm tắt"""
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

def summarize_long_text(text):
    """Tóm tắt nội dung dài bằng cách chia nhỏ và tổng hợp"""
    chunks = chunk_text(text)
    summaries = [summarize_text(chunk) for chunk in chunks]
    return " ".join(summaries)

def process_document(row):
    """Tóm tắt một dòng dữ liệu"""
    raw_text = extract_text(row["Content"])
    summary = summarize_long_text(raw_text)

    return {
        "title": row["Title"],
        "inner_title": row["Inner Title"],
        "link": row["Link"],
        "summary": summary
    }

def process_all_data(num_workers=4):
    """Xử lý toàn bộ dữ liệu và lưu kết quả"""
    data = pd.read_csv('courses_data/scikit_learn.csv')
    
    with Pool(num_workers) as p:  # Chạy đa luồng
        results = p.map(process_document, data)

    df = pd.DataFrame(results)
    df.to_csv('out.csv', index=False, encoding="utf-8-sig")

    print(f"✅ Đã lưu kết quả vào")

if __name__ == "__main__":
    process_all_data()


🔹 Đang chạy trên: cpu


Device set to use cpu
